## Azure SQL Database

In [19]:
from sqlalchemy import create_engine
import pandas as pd
import pyspark as ps
import getpass

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [15]:
print(f'pandas   version: {pd.__version__}')

pandas   version: 2.1.3


## DB Connection

In [31]:
import pyodbc
# from sqlalchemy import create_engine
# import pandas as pd

user = ""
pwd = ""
dbname = ""
host = ".database.windows.net"
port = 

driver = "ODBC Driver 17 for SQL Server"

connection_string = f"Driver={driver};Server={host};Database={dbname};Uid={user};Pwd={pwd};Port={port}"

# engine = create_engine(connection_string)

connection = pyodbc.connect(connection_string)

In [ ]:
# Not in use

user=getpass.getpass('enter Azure SQL user name')
pwd=getpass.getpass('enter Azure SQL password')

In [ ]:
# Not in use

driver = 'ODBC Driver 17 for SQL Server'.replace(' ', '+')
dbname=''
host='.database.windows.net'
port=

con = create_engine(f'mssql+pyodbc://{user}:{pwd}@{host}/{dbname}?driver={driver}')

In [ ]:
!pip install mssql

## All data management commands

In [20]:
# Create a cursor and show all table name

cursor = connection.cursor()

cursor.execute("SELECT name FROM sys.tables")

table_names = [row[0] for row in cursor.fetchall()]

print(table_names)

['regions', 'countries', 'locations', 'jobs', 'departments', 'employees', 'dependents']


In [23]:
cursor.execute("select * from regions")

# Fetch the first 5 rows
data = cursor.fetchmany(5)

data

[(1, 'Europe'), (2, 'Americas'), (3, 'Asia'), (4, 'Middle East and Africa')]

In [26]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

df = pd.read_sql("select * from regions", connection)

df = df.iloc[:5]

print(df)

   region_id             region_name
0          1                  Europe
1          2                Americas
2          3                    Asia
3          4  Middle East and Africa


In [28]:
df = pd.read_sql("select * from regions", connection)

df = df.iloc[:5]

print(df)

   region_id             region_name
0          1                  Europe
1          2                Americas
2          3                    Asia
3          4  Middle East and Africa


In [35]:
df = pd.read_sql("select * from countries", connection)

# df = df.iloc[:5]

print(df)

   country_id              country_name  region_id
0          AR                 Argentina          2
1          AU                 Australia          3
2          BE                   Belgium          1
3          BR                    Brazil          2
4          CA                    Canada          2
5          CH               Switzerland          1
6          CN                     China          3
7          DE                   Germany          1
8          DK                   Denmark          1
9          EG                     Egypt          4
10         FR                    France          1
11         HK                  HongKong          3
12         IL                    Israel          4
13         IN                     India          3
14         IT                     Italy          1
15         JP                     Japan          3
16         KW                    Kuwait          4
17         MX                    Mexico          2
18         NG                  

In [36]:
df = pd.read_sql("select * from jobs", connection)

print(df)

    job_id                        job_title  min_salary  max_salary
0        1                Public Accountant      4200.0      9000.0
1        2               Accounting Manager      8200.0     16000.0
2        3         Administration Assistant      3000.0      6000.0
3        4                        President     20000.0     40000.0
4        5    Administration Vice President     15000.0     30000.0
5        6                       Accountant      4200.0      9000.0
6        7                  Finance Manager      8200.0     16000.0
7        8   Human Resources Representative      4000.0      9000.0
8        9                       Programmer      4000.0     10000.0
9       10                Marketing Manager      9000.0     15000.0
10      11         Marketing Representative      4000.0      9000.0
11      12  Public Relations Representative      4500.0     10500.0
12      13                 Purchasing Clerk      2500.0      5500.0
13      14               Purchasing Manager     

## Backup

In [ ]:
con.table_names()

In [ ]:
pd.read_sql("select top 5 * from disney_char", con)

In [ ]:
df=pd.read_sql("select top 5 * from disney_char", con)
df

In [ ]:
pd.read_sql("select top 5 * from movie_gross", con)

In [ ]:
df=pd.read_sql("""
select year(release_date) year, count(*) [movies] 
    from movie_gross 
    where year(release_date) > 2010
    group by year(release_date)
""", con)
df

In [ ]:
sns.lineplot(x=df.year, y=df.movies)

In [ ]:
pd.read_sql("select top (?) * from disney_char where year(release_date) > ?", con, params=[10, 2010])

In [ ]:
def run_sql(sql: str, params=None):
    return pd.read_sql(sql, con, params=params)

In [ ]:
run_sql("""
select top 5 * 
    from disney_char 
""")

In [ ]:
run_sql("""
select top 5 *
    from disney_char 
    where movie_title = 'Bambi'
""")

In [ ]:
run_sql("""
select top 5 *
    from disney_char 
    where movie_title = ?
""", ['Bambi'])

In [ ]:
run_sql("""
select top 5 * 
    from disney_char 
    where lower(movie_title) like '%%frog%%'
""")

In [ ]:
run_sql("""
select * 
    from disney_char 
    where lower(movie_title) like ?
""", ['%frog%'])

In [ ]:
run_sql("select top 5 * from movie_gross")

In [ ]:
run_sql("""
select top (?) * 
    from movie_gross 
    order by inflation_adjusted_gross desc
""", [10])

In [ ]:
df=run_sql("""
select genre, count(*) [# movies] from movie_gross group by genre order by [# movies] desc
""")
df